In [ ]:
import csv
import numpy as np
from shutil import copy2
import os

In [17]:
# Load result from csv

# Parameters
class_list = ['Cloud','Crystal','Feather','Twinning_wisp']
csv_path = '/media/hdd/lingjia/hdd_diamond/cls/result/2022-11-04-16-18-18_resnet101_resize_plain.csv'


class_list_v0 = ['Cloud', 'Crystal', 'Feather', 'Needle', 'Twinning_wisp']

with open(csv_path, newline='') as f:
    reader = csv.reader(f)
    infer_result = list(reader)

img_dict = {}
for l in infer_result[1:]:
    img_name = l[0]
    gt_cls = l[1]
    # gt_idx = int(l[2])
    gt_idx = class_list.index(gt_cls)
    gt_idx_v0 = class_list_v0.index(gt_cls)
    pred_idx = l[3:].index(max(l[3:]))
    pred_cls = class_list[pred_idx]
    img_dict[img_name] = [img_name, gt_cls, gt_idx, gt_idx_v0, pred_cls, pred_idx]
print(img_dict[img_name])



['10363313859_76.png', 'Twinning_wisp', 3, 4, 'Cloud', 0]


In [2]:
""" VIS1: Save to GT_Pred Folders """
# Parameter
data_path = '/media/hdd/lingjia/hdd_diamond/cls/data/cls5/exp1/test'
save_path = '/media/hdd/lingjia/hdd_diamond/cls/weightedsampler_resnet101_1vsall/patch_vis'


for key in img_dict.keys():
    temp_from = os.path.join(data_path,img_dict[key][1],key)
    temp_to = os.path.join(save_path, img_dict[key][1]+'_'+img_dict[key][3])
    os.makedirs(temp_to,exist_ok=True)
    copy2(temp_from, os.path.join(temp_to,key))


In [18]:
""" Confusion Matrix """
np.set_printoptions(suppress=True)

# show all gt class
C = np.zeros((len(class_list_v0),len(class_list)))
for key in img_dict.keys():
    row = img_dict[key][3]
    col = img_dict[key][5]
    C[row,col] += 1

print(f'v1:\n{C}')

C = np.zeros((len(class_list),len(class_list)))
for key in img_dict.keys():
    row = img_dict[key][2]
    col = img_dict[key][5]
    C[row,col] += 1

print(f'v2:\n{C}')


""" Accuracy """
for row in range(C.shape[0]):
    acc = C[row,row]/np.sum(C[row,:])
    print(f'{class_list[row]} {int(C[row,row])}/{int(np.sum(C[row,:]))}={acc*100:.2f}%')
    

"""
val:
[[1420.  402.]
 [ 381. 1336.]]
Crystal 77.94%
other 77.81%

test:
[[1422.  400.]
 [ 387. 1330.]]
Crystal 78.05%
other 77.46%
"""

v1:
[[ 810.  481.   31.   16.]
 [ 236. 1384.   22.  179.]
 [  37.   77.   20.    4.]
 [   0.    0.    0.    0.]
 [ 119.   52.    3.   10.]]
v2:
[[ 810.  481.   31.   16.]
 [ 236. 1384.   22.  179.]
 [  37.   77.   20.    4.]
 [ 119.   52.    3.   10.]]
Cloud 810/1338=60.54%
Crystal 1384/1821=76.00%
Feather 20/138=14.49%
Twinning_wisp 10/184=5.43%


'\nval:\n[[1420.  402.]\n [ 381. 1336.]]\nCrystal 77.94%\nother 77.81%\n\ntest:\n[[1422.  400.]\n [ 387. 1330.]]\nCrystal 78.05%\nother 77.46%\n'

In [6]:
""" VIS2 show on 1 complete diamond image """
# Parameters
txt_path = '/media/hdd/lingjia/hdd_diamond/cls/data/plain_v3/txt_file'
img_path = '/media/hdd/diamond/data/imgs'
vis_save_path = '/media/hdd/lingjia/hdd_diamond/cls/weightedsampler_resnet101_1vsall/vis/Twinning_wisp'
os.makedirs(vis_save_path,exist_ok=True)

# Start
from collections import defaultdict
import cv2
font = cv2.FONT_HERSHEY_SIMPLEX
from  matplotlib import pyplot as plt

# diamond IDs
diamond_ids = defaultdict(list)
for key in img_dict.keys():
    diamond_ids[key[:11]].append(img_dict[key])

# diamond_id = '10381382349'
# print(diamond_ids[diamond_id])
for diamond_id in diamond_ids.keys():
    if '10377900198' in diamond_id or '10377322674' in diamond_id:
        continue

    with open(os.path.join(txt_path,diamond_id+'.txt')) as f:
        txt_file = f.readlines()
    txt_dict = {}
    for t in txt_file:
        pname = t.split(' ')[0]
        pcls = t.split(' ')[2]
        coords = t[:-1].split('(')[-1][:-1]
        txt_dict[pname] = [pname,pcls,coords]

    # open Image
    img = cv2.imread(os.path.join(img_path,diamond_id+'.png'))
    # plt.imshow(img)
    # plt.show()

    for patch in diamond_ids[diamond_id]:
        pname = patch[0]
        assert patch[1] == txt_dict[pname[:-4]][1]
        coords = txt_dict[pname[:-4]][2]
        coords = [int(c) for c in coords.split(', ')]

        # if TP
        if patch[1] == patch[4]:
            color = (0,255,0)
            # tt = pname[11:-4]+' '+str(patch[2])
            tt = pname[12:-4]+' '+str(patch[3])
        else:
            color = (0,0,255)
            tt = pname[12:-4]+' '+str(patch[3]) #+' '+str(patch[5])

        cv2.rectangle(img, (coords[0], coords[1]), (coords[2], coords[3]), color, 2)
        cv2.putText(img, tt, (coords[0], coords[1]-5), font, 0.3, color, 1, cv2.LINE_AA)

    # plt.imshow(img)
    # plt.show()
    cv2.imwrite(os.path.join(vis_save_path,diamond_id+'.png'),img)
    # print(f'{diamond_id} finish')


In [30]:
# Move images to one single folder

img_path = '/media/hdd/diamond/data/hdr_data_v2'
save_path = '/media/hdd/diamond/data/imgs'
os.makedirs(save_path,exist_ok=True)

for r,d,f in os.walk(img_path):
    for ff in f:
        if 'png' in ff and 'mask' not in ff:
            copy2(os.path.join(r,ff),os.path.join(save_path,ff[:11]+'.png'))

